In [49]:
# Gravitational Lensing Research
# Program to take a qlens-generated sample chain and derive alternative parameters
# d3: This version takes the cnfw pmode=1 format (mvir, c, beta) and derives scale radius (r_s),
# cnfw param r_c (r_c), 'core radius' (r_core, defined as where log slope=-1) and revised concentration (c_hat)
# c_hat_200 is the defined as r_vir / r_{-2}, and r_{-2} is where the log slope is -2.
# Uses python 3.6
# input from cNFW needed is mvir, c, rc_kpc

In [50]:
import time
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import pandas as pd
import timeit
from IPython.core.debugger import set_trace
import datetime
from shutil import copyfile
from scipy.optimize import minimize_scalar

In [51]:
# The current working directory should CONTAIN the "chains_..." subdirectory for the run of interest.
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/A611_temp/chains_Abell_611_v30.ip.cNFW_d5


In [52]:
label = 'RXCJ2248.cnfw.v8'  # Don't use this on NFW, there is no 'beta' parameter.
path = '/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/RXCJ2248' 
os.chdir(path)
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/RXCJ2248


In [53]:
# verify that the key files exist
assert os.path.exists('chains_' + label)
for ext in ['', '.nparam', '.paramnames', '.latex_paramnames', '.ranges']:
    assert os.path.exists('chains_' + label + '/' + label + ext)

In [54]:
# create the new directory, if it doesn't already exist
app = '_d5'
newlabel = label + app
if not os.path.exists(os.getcwd() + '/chains_' + newlabel):
        os.makedirs(os.getcwd() + '/chains_' + newlabel)

In [55]:
# copy key files
for ext in ['', '.nparam', '.paramnames', '.latex_paramnames', '.ranges']:
    copyfile('chains_' + label + '/' + label + ext, 'chains_' + newlabel + '/' + newlabel + ext) 

In [56]:
# create 'newpath', for convenience
newpath = path + '/chains_' + newlabel

In [57]:
# we are deriving scale radius and core radius
addl_pnames = ['r_s', 'r_c', 'r_core', 'c_hat_200', 'rm2']
addl_latex_pnames = ['r_{s}', 'r_{c}', 'r_{core}', '\hat{c}_{200}', 'r_{-2}']

In [58]:
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s /u.Mpc, Om0=0.3)
cosmo

FlatLambdaCDM(H0=70 km / (Mpc s), Om0=0.3, Tcmb0=0 K, Neff=3.04, m_nu=None, Ob0=None)

In [59]:
zlens = 0.288

In [60]:
rho_crit = cosmo.critical_density(zlens).to(u.M_sun / u.kpc**3)
print(rho_crit)
kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(zlens) * u.arcmin / (60. * u.arcsec)
print (kpc_per_arcsec)
rho_200 = rho_crit * 200 * kpc_per_arcsec**3  # units of M_sun arcsec^-3
print('rho_200 = ', rho_200)
# get rid of units now
rho_200 = rho_200.value

182.36871306056253 solMass / kpc3
4.329314910896306 kpc / arcsec
rho_200 =  2959632.997002065 solMass / arcsec3


In [61]:
# read in the chain and append the new parameter names
samples = np.loadtxt(newpath + '/' + newlabel, comments="#", delimiter=None, unpack=False)
with open(newpath + '/' + newlabel + '.paramnames') as afile:
    paramnames = afile.readlines()
with open(newpath + '/' + newlabel + '.latex_paramnames') as afile:
    latex_paramnames = afile.readlines()
# append the new parameter names
for nm, latex_nm in zip(addl_pnames, addl_latex_pnames):
    paramnames.append(nm + '\n')
    latex_paramnames.append(nm + '\t' + latex_nm + '\n')
    with open(newpath + '/' + newlabel + '.paramnames', 'a') as afile:
        afile.write(nm + '\n')
    with open(newpath + '/' + newlabel + '.latex_paramnames', 'a') as afile:
        afile.write(nm + '\t' + latex_nm + '\n')

In [62]:
# change the working directory to the new path
os.chdir(newpath)
print(os.getcwd())

/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/RXCJ2248/chains_RXCJ2248.cnfw.v8_d5


In [63]:
# increase the number of derived parameters in the .nparam file
pnum, dpnum = np.loadtxt(newlabel + '.nparam', comments="#", delimiter=None, unpack=False)
addl_np = len(addl_pnames)
print(dpnum, ' derived parameters, adding ', addl_np)
dpnum += addl_np
np.savetxt(newlabel + '.nparam', [np.int(pnum), np.int(dpnum)], fmt='%i')

# add new ranges to the .ranges file. Not sure if these should be calculated, just using 0 and 1e30.
with open(newlabel + '.ranges', 'a') as afile:
    for nm in addl_pnames:
        afile.write('0  1e30\n')

# Strip off the \n character.
paramnames = [item[:-1] for item in paramnames] 
latex_paramnames = [item[:-1] for item in latex_paramnames]  
samples = np.array(samples)

1.0  derived parameters, adding  5


In [64]:
ns, p = samples.shape # note that 'p' here is two more than the number of parameters.
# as it counts the first column (weight) and the last (chisquare).
print(samples.shape)
print(paramnames)

(95405, 14)
['mvir', 'c', 'rc_kpc', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2', 'mtot1', 'mtot2', 'raw_chisq', 'r_s', 'r_c', 'r_core', 'c_hat_200', 'rm2']


In [65]:
# check to make sure the names are there. Also finds log(paramname).
assert all([any([name.find(item)!=-1 for item in ['mvir', 'c', 'rc_kpc']]) for name in paramnames[0:3]])

In [66]:
# extend the samples to have addl_np additional columns
samples2 = np.append(samples, np.zeros((ns, addl_np)), axis=1)

In [67]:
# move the chisquare to be the last column
samples2[:,-1] = samples2[:, p-1]

In [68]:
# calculate the new parameters
target_cols = [1, 2, 3]
targets = [col for col in samples2[:, target_cols].T] 
# for the cc model with n constant, we need cols [1,2, 3, 4, 5], which are k0, gamma, a1, s and q
# for the pmode=3 cNFW model we need cols [1, 2, 3], which are mvir, c, and rc_kpc.

In [69]:
# convert log parameters to non-log. But we are not saving them. Let mkdist do the tranform for plots, as
# you need to change the latex names.
for i,pm in enumerate(targets):
    if paramnames[i][0:4] == 'log(':
        targets[i] = 10** pm
#         samples[:,i] = 10** samples[:,i]
#         paramnames[i-1] = paramnames[i-1][4:len(paramnames[i-1])-1]
        
# print(paramnames)        

In [70]:
mvir, c, rc_kpc = targets

In [71]:
mvir, c, rc_kpc

(array([9.22921e+14, 1.33811e+15, 9.66055e+14, ..., 1.68365e+15,
        1.76934e+15, 1.63059e+15]),
 array([12.4721 , 13.5587 , 12.6208 , ...,  7.01562,  6.36054,  7.47692]),
 array([ 85.6659, 171.578 ,  84.6951, ...,  54.4686,  46.7505,  60.8804]))

In [72]:
np.amin(rc_kpc)

0.00448616

We can use the equation for virial mass and solve for the virial radius:
$m_{200} = \frac{4}{3} \pi r_{200}^3 \rho_{200}$, and $r_s = r_{200} / c$,
Therefore
$$ r_s = (\frac{3}{4\pi})^{1/3} (m_{200} / (c  \rho_{200}))^{1/3}$$

In [73]:
(3 / (np.pi * 4.))**(1./3)

0.6203504908994001

In [74]:
# Now need a function to find c_hat. First, the cnfw log slope.
def cnfwlogslope(r, rs, rc): 
    return - (r * (3 * r + 2* rc + rs)) / ((r + rc) * (r + rs))
# test:
cnfwlogslope(15, 15, 0)

-2.0

In [75]:
# A function to find r_{beta}, the radius at which the log slope is 'beta'.
def rbeta_cnfw(r_s, r_c, beta):
    if beta == -1:
        return r_c * (np.sqrt(1 + 8. * r_s / r_c) - 1) / 4.
    elif beta == -2:
        return r_s * (np.sqrt(1 + 8. * r_c / r_s) + 1) / 2.
    else:
        rb = minimize_scalar(lambda r: np.abs(beta - cnfwlogslope(r, r_s, r_c)), method='bounded', bounds=(0,5000))
        if rb.success !=True:
            print('Optimize Failure')
            print(rb)
            print(r_s, r_c, beta)
        return rb.x
# test
print(rbeta_cnfw(36.8, 24.5, -2)) # should be 64.679 per Mathematica
print(rbeta_cnfw(20., 12., -1))# should be 8.358

64.67915297409839
8.357816691600547


In [76]:
# calculate the new parameters
rvir = (3 / (np.pi * 4.))**(1./3) * mvir**(1./3) / (rho_200**(1./3))
r_s = rvir / c
r_c = rc_kpc / kpc_per_arcsec.value
beta = r_s / r_c
rm2 = [rbeta_cnfw(r_sp, r_cp, -2) for r_sp, r_cp in zip(r_s, r_c)]
c_hat = rvir / rm2 
r_core = [rbeta_cnfw(r_sp, r_cp, -1) for r_sp, r_cp in zip(r_s, r_c)]

In [77]:
samples2[:, p-1:p-1+addl_np] = np.array([r_s, r_c, r_core, c_hat, rm2]).T

In [78]:
samples2.shape

(95405, 19)

In [79]:
print(paramnames)
print(len(paramnames))

['mvir', 'c', 'rc_kpc', 'q', 'theta', 'xc', 'yc', 'shear1', 'shear2', 'mtot1', 'mtot2', 'raw_chisq', 'r_s', 'r_c', 'r_core', 'c_hat_200', 'rm2']
17


In [80]:
# write the revised samples back to the file
np.savetxt(newlabel, samples2, fmt="%.6g", delimiter='   ')

In [81]:
for i, name in enumerate(paramnames):
    if name == 'raw_chisq':
        print('Best fit chi squared is ', np.amin(samples2[:,i+1]))

Best fit chi squared is  70.1092


In [82]:
newlabel

'RXCJ2248.cnfw.v8_d5'

In [83]:
sys.exit()

SystemExit: 

/Users/kevin/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [88]:
# If there is a transform that you want to make, and a dat file for conversion, specify it here
transform = True
t_file = 'rcx2_transform.dat'

In [89]:
# make the error file using mkdist
if transform:
    code = os.system('mkdist ' + newlabel + ' -E -T:../' + t_file + ' | tee ' + newlabel + '.err')
else:
    code = os.system('mkdist ' + newlabel + ' | tee ' + newlabel + '.err')    
if code != 0:
    print('Error! Code returned ', code)
else:
    print('Success')

Success


In [90]:
# make the various distributions and plots
# having trouble with 'ranges much smaller than actual ranges'
# so try adding the option '-B5e-4'
if transform:
    code = os.system('mkdist ' + newlabel + ' -n40 -N40  -B5e-4 -T:../' + t_file + ' |tee tmp.txt')
else:
    code = os.system('mkdist ' + newlabel + ' -n40 -N40  -B5e-4'+ ' |tee tmp.txt')
if code != 0:
    print('Error! Code returned ', code)
else:
    with open('tmp.txt', 'r') as file:
        tmp = file.readlines()
    if 'WARNING' in tmp or 'Error:' in tmp:
        print('Completed with WARNINGS or Error')
    elif len(tmp) == 1:
        print('Warning: output was only one line:')
        print(tmp)
    else:
        print('Success')

Success


In [ ]:
sys.exit()

In [91]:
# to run the python triangle plot pdf maker
code = os.system('python ' + newlabel + '_tri.py')
if code != 0:
    print('Error! Code returned ', code)
else:
    print('Success')

Success
